In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [17]:
# This will collect the links for all cities on craigslist. We will have to sparse through
# these and cut out the non-US cities.
def city_link_collector():
    # this is the craigslist page with every city
    main_page = requests.get('https://www.craigslist.org/about/sites')
    soup = BeautifulSoup(main_page.text, 'html.parser')
    
    all_list = []
    uscity_list = []
    city_list = []
    
    for i in range(4):
        for box in soup.find_all('div', class_='box box_{}'.format(i+1)):
            all_list.append(box.find_all('a'))
    for _ in all_list[:20:7]:
        for __ in _:
            uscity_list.append(__)
    for ___ in all_list[20][0:94]:
        uscity_list.append(___)
    
    for idx, city in enumerate(uscity_list):
        city_list.append(str(uscity_list[idx]).split('''"''')[1])
    
    '''These listings are all a subset of Miami and break the scraper if entered in this format'''
    city_list.remove('http://miami.craigslist.org/brw/')
    city_list.remove('http://miami.craigslist.org/mdc/')
    city_list.remove('http://miami.craigslist.org/pbc/')
    '''Here is the Miami list to capture the dropped entries and will not break the scraper'''
    city_list.append('http://miami.craigslist.org/')
    return city_list
    #                     #posting date
    #                     #grab the datetime element 0 for date and 1 for time
    #                     post_datetime = box.find('time', class_= 'result-date')['datetime']
    #                     post_timing.append(post_datetime)

            

In [18]:
city_list = city_link_collector()
len(city_list)

415

In [2]:
def craigslist_motorcycle_scraper(city_list):

    title = []
    price = []
    neighborhood = []
    attributes = []
    description = []
    city_names = []

    for city in city_list: 

        #get the first page of the Austin motorcycle prices
        response = requests.get('{}search/mca?s=0&bundleDuplicates=1'.format(city))
        #parse through it and make it readable
        html_soup = BeautifulSoup(response.text, 'html.parser')
        #find the total number of posts to find the limit for each page
        results_num = html_soup.find('div', class_= 'search-legend')
        #pulled the total count of posts as the upper bound of the pages array
        results_total = int(results_num.find('span', class_='totalcount').text) 
        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        print("{} Posts = {}".format(city, results_total))
        print("{} Pages = {}".format(city.title(), len(pages)))

        iterations = 0

        for page in pages:         
            title_ = []
            price_ = []
            neighborhood_ = []
            attributes_ = []
            description_ = []
            city_names_ = []
            #get request      
            response = requests.get("{}search/mca?".format(city) 
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&bundleDuplicates=1")

            time.sleep(random.randint(2,3))

            #throw warning for status codes that are not 200
            if response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, response.status_code))

            #define the html text
            post_soup = BeautifulSoup(response.text, 'html.parser')

            #define the posts
    #         posts = page_html.find_all('li', class_= 'result-row')

            #extract data by item
    #         for post in posts:
            count = 0
            for post in post_soup.find_all('a', class_ = 'result-title hdrlnk'):
                link = post['href']
                sub_post = requests.get(link)
                sub_soup = BeautifulSoup(sub_post.text, 'html.parser')

    #             if tag.find('small') is not None:

                post_title = sub_soup.find('span', id = 'titletextonly')
                title_.append(post_title)

                post_price = sub_soup.find('span', class_ = 'price')
                price_.append(post_price)

                post_neighborhood = sub_soup.find('small')
                neighborhood_.append(post_neighborhood)

                post_attributes = sub_soup.find_all('p', attrs = {'class': 'attrgroup'})
                attributes_.append(post_attributes)

                post_description = sub_soup.find('section', id = 'postingbody')
                description_.append(post_description)

                city_names_.append(city)

                time.sleep(random.randint(2,3))
                count += 1
                
                if count == results_total:
                    break

            iterations += 1
            print("{} Page ".format(city.title()) + str(iterations) + " of {} pages".format(len(pages)) + " scraped successfully!")

            title.append(title_)
            price.append(price_)
            neighborhood.append(neighborhood_)
            attributes.append(attributes_)
            description.append(description_)
            city_names.append(city_names_)


        print("\n")
        print("{} complete!".format(city.title()))
        print(str(len(title_)) + " rows collected.")
        print("\n")
    return title, price, neighborhood, attributes, description, city_names

'https://phoenix.craigslist.org/'

In [88]:
%time titleto15, priceto15, neighborhoodto15, attributesto15, descriptionto15, cityto15 = craigslist_motorcycle_scraper(city_list[0:15])

https://auburn.craigslist.org/ Posts = 7
Https://Auburn.Craigslist.Org/ Pages = 1
Https://Auburn.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Auburn.Craigslist.Org/ complete!
7 rows collected.


https://bham.craigslist.org/ Posts = 108
Https://Bham.Craigslist.Org/ Pages = 1
Https://Bham.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Bham.Craigslist.Org/ complete!
108 rows collected.


https://dothan.craigslist.org/ Posts = 22
Https://Dothan.Craigslist.Org/ Pages = 1
Https://Dothan.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Dothan.Craigslist.Org/ complete!
22 rows collected.


https://shoals.craigslist.org/ Posts = 9
Https://Shoals.Craigslist.Org/ Pages = 1
Https://Shoals.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Shoals.Craigslist.Org/ complete!
9 rows collected.


https://gadsden.craigslist.org/ Posts = 10
Https://Gadsden.Craigslist.Org/ Pages = 1
Https://Gadsden.Craigslist.Org/ Page 1 of 1 pages scr

In [89]:
%time title16to30, price16to30, neighborhood16to30, attributes16to30, description16to30, city16to30 = craigslist_motorcycle_scraper(city_list[16:31])

https://prescott.craigslist.org/ Posts = 45
Https://Prescott.Craigslist.Org/ Pages = 1
Https://Prescott.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Prescott.Craigslist.Org/ complete!
45 rows collected.


https://showlow.craigslist.org/ Posts = 15
Https://Showlow.Craigslist.Org/ Pages = 1
Https://Showlow.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Showlow.Craigslist.Org/ complete!
15 rows collected.


https://sierravista.craigslist.org/ Posts = 8
Https://Sierravista.Craigslist.Org/ Pages = 1
Https://Sierravista.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Sierravista.Craigslist.Org/ complete!
8 rows collected.


https://tucson.craigslist.org/ Posts = 519
Https://Tucson.Craigslist.Org/ Pages = 5
Https://Tucson.Craigslist.Org/ Page 1 of 5 pages scraped successfully!
Https://Tucson.Craigslist.Org/ Page 2 of 5 pages scraped successfully!
Https://Tucson.Craigslist.Org/ Page 3 of 5 pages scraped successfully!
Https://Tucson.Crai

In [90]:
%time title31to50, price31to50, neighborhood31to50, attributes31to50, description31to50, city31to50 = craigslist_motorcycle_scraper(city_list[31:51])

https://humboldt.craigslist.org/ Posts = 65
Https://Humboldt.Craigslist.Org/ Pages = 1
Https://Humboldt.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Humboldt.Craigslist.Org/ complete!
65 rows collected.


https://imperial.craigslist.org/ Posts = 8
Https://Imperial.Craigslist.Org/ Pages = 1
Https://Imperial.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Imperial.Craigslist.Org/ complete!
8 rows collected.


https://inlandempire.craigslist.org/ Posts = 847
Https://Inlandempire.Craigslist.Org/ Pages = 8
Https://Inlandempire.Craigslist.Org/ Page 1 of 8 pages scraped successfully!
Https://Inlandempire.Craigslist.Org/ Page 2 of 8 pages scraped successfully!
Https://Inlandempire.Craigslist.Org/ Page 3 of 8 pages scraped successfully!
Https://Inlandempire.Craigslist.Org/ Page 4 of 8 pages scraped successfully!
Https://Inlandempire.Craigslist.Org/ Page 5 of 8 pages scraped successfully!
Https://Inlandempire.Craigslist.Org/ Page 6 of 8 pages scraped succes

Https://Slo.Craigslist.Org/ Page 1 of 2 pages scraped successfully!
Https://Slo.Craigslist.Org/ Page 2 of 2 pages scraped successfully!


Https://Slo.Craigslist.Org/ complete!
18 rows collected.


https://santabarbara.craigslist.org/ Posts = 67
Https://Santabarbara.Craigslist.Org/ Pages = 1
Https://Santabarbara.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Santabarbara.Craigslist.Org/ complete!
67 rows collected.


https://santamaria.craigslist.org/ Posts = 24
Https://Santamaria.Craigslist.Org/ Pages = 1
Https://Santamaria.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Santamaria.Craigslist.Org/ complete!
24 rows collected.


https://siskiyou.craigslist.org/ Posts = 5
Https://Siskiyou.Craigslist.Org/ Pages = 1
Https://Siskiyou.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Siskiyou.Craigslist.Org/ complete!
5 rows collected.


https://stockton.craigslist.org/ Posts = 87
Https://Stockton.Craigslist.Org/ Pages = 1
Https://Stockton

In [91]:
%time title15, price15, neighborhood15, attributes15, description15, city15 = craigslist_motorcycle_scraper(city_list[15])

MissingSchema: Invalid URL 'hsearch/mca?s=0&bundleDuplicates=1': No schema supplied. Perhaps you meant http://hsearch/mca?s=0&bundleDuplicates=1?

In [6]:
%time title51to70, price51to70, neighborhood51to70, attributes51to70, description51to70, city51to70 = craigslist_motorcycle_scraper(city_list[51:71])

https://ventura.craigslist.org/ Posts = 205
Https://Ventura.Craigslist.Org/ Pages = 2
Https://Ventura.Craigslist.Org/ Page 1 of 2 pages scraped successfully!
Https://Ventura.Craigslist.Org/ Page 2 of 2 pages scraped successfully!


Https://Ventura.Craigslist.Org/ complete!
85 rows collected.


https://visalia.craigslist.org/ Posts = 59
Https://Visalia.Craigslist.Org/ Pages = 1
Https://Visalia.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Visalia.Craigslist.Org/ complete!
59 rows collected.


https://yubasutter.craigslist.org/ Posts = 27
Https://Yubasutter.Craigslist.Org/ Pages = 1
Https://Yubasutter.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Yubasutter.Craigslist.Org/ complete!
27 rows collected.


https://boulder.craigslist.org/ Posts = 58
Https://Boulder.Craigslist.Org/ Pages = 1
Https://Boulder.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Boulder.Craigslist.Org/ complete!
58 rows collected.


https://cosprings.craigslis

AttributeError: 'NoneType' object has no attribute 'find'

In [7]:
%time title71to100, price71to100, neighborhood71to100, attributes71to100, description71to100, city71to100 = craigslist_motorcycle_scraper(city_list[71:101])

https://fortlauderdale.craigslist.org/ Posts = 885
Https://Fortlauderdale.Craigslist.Org/ Pages = 8
Https://Fortlauderdale.Craigslist.Org/ Page 1 of 8 pages scraped successfully!
Https://Fortlauderdale.Craigslist.Org/ Page 2 of 8 pages scraped successfully!
Https://Fortlauderdale.Craigslist.Org/ Page 3 of 8 pages scraped successfully!
Https://Fortlauderdale.Craigslist.Org/ Page 4 of 8 pages scraped successfully!
Https://Fortlauderdale.Craigslist.Org/ Page 5 of 8 pages scraped successfully!
Https://Fortlauderdale.Craigslist.Org/ Page 6 of 8 pages scraped successfully!
Https://Fortlauderdale.Craigslist.Org/ Page 7 of 8 pages scraped successfully!
Https://Fortlauderdale.Craigslist.Org/ Page 8 of 8 pages scraped successfully!


Https://Fortlauderdale.Craigslist.Org/ complete!
45 rows collected.


https://fortmyers.craigslist.org/ Posts = 243
Https://Fortmyers.Craigslist.Org/ Pages = 3
Https://Fortmyers.Craigslist.Org/ Page 1 of 3 pages scraped successfully!
Https://Fortmyers.Craigslist.Org

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
%time title101to200, price101to200, neighborhood101to200, attributes101to200, description101to200, city101to200 = craigslist_motorcycle_scraper(city_list[101:201])

https://statesboro.craigslist.org/ Posts = 4
Https://Statesboro.Craigslist.Org/ Pages = 1
Https://Statesboro.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Statesboro.Craigslist.Org/ complete!
4 rows collected.


https://valdosta.craigslist.org/ Posts = 16
Https://Valdosta.Craigslist.Org/ Pages = 1
Https://Valdosta.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Valdosta.Craigslist.Org/ complete!
16 rows collected.


https://honolulu.craigslist.org/ Posts = 482
Https://Honolulu.Craigslist.Org/ Pages = 5
Https://Honolulu.Craigslist.Org/ Page 1 of 5 pages scraped successfully!
Https://Honolulu.Craigslist.Org/ Page 2 of 5 pages scraped successfully!
Https://Honolulu.Craigslist.Org/ Page 3 of 5 pages scraped successfully!
Https://Honolulu.Craigslist.Org/ Page 4 of 5 pages scraped successfully!
Https://Honolulu.Craigslist.Org/ Page 5 of 5 pages scraped successfully!


Https://Honolulu.Craigslist.Org/ complete!
2 rows collected.


https://boise.craigslist

In [104]:
title = []
for _ in titleto15:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)
    
for _ in title16to30:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)
    
for _ in title31to50:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)
        
price = []

for _ in priceto15:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)

for _ in price16to30:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)

for _ in price31to50:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)
        
neighborhood = []

for _ in neighborhoodto15:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)
    
for _ in neighborhood16to30:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)
    
for _ in neighborhood31to50:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)
         
attributes = []

for _ in attributesto15:
    for __ in _:
        attributes.append(__)
        
for _ in attributes16to30:
    for __ in _:
        attributes.append(__)
        
for _ in attributes31to50:
    for __ in _:
        attributes.append(__)

description = []

for _ in descriptionto15:
    for __ in _:
        description.append(__)
        
for _ in description16to30:
    for __ in _:
        description.append(__)
        
for _ in description31to50:
    for __ in _:
        description.append(__)
        
city = []

for _ in cityto15:
    for __ in _:
        city.append(__)
        
for _ in city16to30:
    for __ in _:
        city.append(__)
        
for _ in city31to50:
    for __ in _:
        city.append(__)

In [109]:
len(title), len(price), len(neighborhood), len(attributes), len(description), len(city)

(11725, 11725, 11725, 11725, 11725, 11725)

In [110]:
# title = np.asarray(title)
# price = np.asarray(price)
# neighborhood = np.asarray(neighborhood)
# attributes = np.asarray(attributes)
# description = np.asarray(description)
# city = np.asarray(city)

data = {'title': title, 'price': price, 'neighborhood': neighborhood, 'attributes': attributes, 'description': description, 'city': city}
df = pd.DataFrame(data=data)

In [111]:
df.head()

,title,price,neighborhood,attributes,description,city
0,[2016 Honda CBR300r],[$2900],[ (Auburn)],"[[\n, [<b>2016 honda cbr300r</b>], \n, [], \n]...","[\n, [\n, [QR Code Link to This Post], \n, [],...",https://auburn.craigslist.org/
1,"[BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERY...",None,[ (FAYETTEVILLE)],"[[\n, [<b> all bikes</b>], \n, [], \n], [\n, [...","[\n, [\n, [QR Code Link to This Post], \n, [],...",https://auburn.craigslist.org/
2,[2017 Yamaha FZ 07 very low miles],[$5000],[ (Near Montgomery)],"[[\n, [<b>2017 Yamaha FZ-07</b>], \n, [], \n],...","[\n, [\n, [QR Code Link to This Post], \n, [],...",https://auburn.craigslist.org/
3,[2010 Yellow Goldwing GL1800 CSC(see pics/pric...,None,"[\n (, [google map], )\n ]","[[\n, [<b>2010 Honda GL1800 CSC</b>], \n, [], ...","[\n, [\n, [QR Code Link to This Post], \n, [],...",https://auburn.craigslist.org/
4,[2006 Honda XR650L *Price Drop*],[$3800],[ (Auburn)],"[[\n, [<b>2006 honda xr650l</b>], \n, [], \n],...","[\n, [\n, [QR Code Link to This Post], \n, [],...",https://auburn.craigslist.org/


In [113]:
df.to_csv('motorcycle_toy_df')

In [35]:
description[:5]

[<section id="postingbody">
 <div class="print-information print-qrcode-container">
 <p class="print-qrcode-label">QR Code Link to This Post</p>
 <div class="print-qrcode" data-location="https://auburn.craigslist.org/mcy/d/auburn-university-2016-honda-cbr300r/7120429265.html"></div>
 </div>
 2016 Honda CBR300r<br/>
 6,186 miles as of listing<br/>
 after market: fender eliminator kit and bubble windscreen<br/>
 I have the original parts if you would prefer those.<br/>
 clean title<br/>
 <br/>
 This is an excellent starter bike or daily commuter. 50+ mpg around auburn and the best parking on campus!<br/>
 <br/>
 Not looking to trade<br/>
 <br/>
 </section>,
 <section id="postingbody">
 <div class="print-information print-qrcode-container">
 <p class="print-qrcode-label">QR Code Link to This Post</p>
 <div class="print-qrcode" data-location="https://auburn.craigslist.org/mcd/d/fayetteville-bad-credit-no-credit-ok-we/7119776172.html"></div>
 </div>
 WE SHIP NATIONWIDE, FINANCE NATIONWIDE! 

In [36]:
city[:5]

['https://auburn.craigslist.org/',
 'https://auburn.craigslist.org/',
 'https://auburn.craigslist.org/',
 'https://auburn.craigslist.org/',
 'https://auburn.craigslist.org/']